<a href="https://colab.research.google.com/github/ishika9/Stack-Overflow-tag-prediction/blob/main/stackoverflow_tag_prediction_colab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## imports

In [1]:
import numpy as np
import pandas as pd
import nltk
import string
import bs4
from bs4 import BeautifulSoup

## reading data

In [3]:
ques=pd.read_csv('/content/drive/MyDrive/stackoverflow tag/Questions.csv', encoding='latin')
tags=pd.read_csv('/content/drive/MyDrive/stackoverflow tag/Tags.csv', encoding='latin')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
ques.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [5]:
tags.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [6]:
ques.shape

(1264216, 7)

In [7]:
tags.shape

(3750994, 2)

In [8]:
ques.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264216 entries, 0 to 1264215
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Id            1264216 non-null  int64  
 1   OwnerUserId   1249762 non-null  float64
 2   CreationDate  1264216 non-null  object 
 3   ClosedDate    55959 non-null    object 
 4   Score         1264216 non-null  int64  
 5   Title         1264216 non-null  object 
 6   Body          1264216 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 67.5+ MB


In [9]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3750994 entries, 0 to 3750993
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Id      int64 
 1   Tag     object
dtypes: int64(1), object(1)
memory usage: 57.2+ MB


## merging dataframes

In [10]:
tags['Tag']=tags['Tag'].astype(str)
grouped_tags=tags.groupby('Id')['Tag'].apply(lambda tags: ' '.join(tags))
grouped_tags.head()

Id
80                            flex actionscript-3 air
90       svn tortoisesvn branch branching-and-merging
120                               sql asp.net sitemap
180    algorithm language-agnostic colors color-space
260           c# .net scripting compiler-construction
Name: Tag, dtype: object

In [11]:
x=grouped_tags


In [12]:
grouped_tags=x
#so lambda doesnt need to be run again

In [13]:
grouped_tags.reset_index()

,Id,Tag
0,80,flex actionscript-3 air
1,90,svn tortoisesvn branch branching-and-merging
2,120,sql asp.net sitemap
3,180,algorithm language-agnostic colors color-space
4,260,c# .net scripting compiler-construction
...,...,...
1264211,40143210,php .htaccess
1264212,40143300,google-bigquery
1264213,40143340,android android-studio
1264214,40143360,javascript vue.js


In [14]:
grouped_tags_final= pd.DataFrame({'Id': grouped_tags.index, 'Tags': grouped_tags.values})
grouped_tags_final

,Id,Tags
0,80,flex actionscript-3 air
1,90,svn tortoisesvn branch branching-and-merging
2,120,sql asp.net sitemap
3,180,algorithm language-agnostic colors color-space
4,260,c# .net scripting compiler-construction
...,...,...
1264211,40143210,php .htaccess
1264212,40143300,google-bigquery
1264213,40143340,android android-studio
1264214,40143360,javascript vue.js


## Data Cleaning and filtering

In [15]:
ques.drop(columns=['OwnerUserId','CreationDate', 'ClosedDate'], inplace=True)


In [16]:
ques=ques.merge(grouped_tags_final, on= 'Id')
ques.head()

,Id,Score,Title,Body,Tags
0,80,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,flex actionscript-3 air
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,svn tortoisesvn branch branching-and-merging
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,sql asp.net sitemap
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,algorithm language-agnostic colors color-space
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,c# .net scripting compiler-construction


In [17]:
print(ques['Score'].min(), ques["Score"].max())

-73 5190


In [18]:
ques=ques[ques['Score']>5]

In [19]:
ques_df=ques.copy()
# would've been a view if we didnt use copy

In [21]:
ques_df['Tags'].astype(str)

0                                    flex actionscript-3 air
1               svn tortoisesvn branch branching-and-merging
2                                        sql asp.net sitemap
3             algorithm language-agnostic colors color-space
4                    c# .net scripting compiler-construction
                                 ...                        
1262915             python pandas dataframe categorical-data
1263065               python iterator nested-loops itertools
1263399                                            f# vscode
1263454                                                  c++
1264205    haskell types functional-programming agda lamb...
Name: Tags, Length: 72950, dtype: object

In [28]:
# ques_df['Tags'].isnull().sum()
ques_df['Tags']

0                                    flex actionscript-3 air
1               svn tortoisesvn branch branching-and-merging
2                                        sql asp.net sitemap
3             algorithm language-agnostic colors color-space
4                    c# .net scripting compiler-construction
                                 ...                        
1262915             python pandas dataframe categorical-data
1263065               python iterator nested-loops itertools
1263399                                            f# vscode
1263454                                                  c++
1264205    haskell types functional-programming agda lamb...
Name: Tags, Length: 72950, dtype: object

In [29]:
ques_df['Tags']=ques_df['Tags'].apply(lambda x: x.split())
flat_tags=[item for sublist in ques_df['Tags'] for item in sublist]
len(flat_tags)

224129

In [31]:
keywords=nltk.FreqDist(flat_tags)
# print(keywords)
word_freq= keywords.most_common(50)
# print(word_freq)
tags_features= [word[0] for word in word_freq]

In [32]:
def most_common(tags):
    tags_filtered=[]
    for i in range(len(tags)):
        if tags[i] in tags_features:
            tags_filtered.append(tags[i])
    return tags_filtered

In [33]:
ques_df['Tags']=ques_df['Tags'].apply(lambda x: most_common(x))
ques_df['Tags']=ques_df['Tags'].apply(lambda x: x if len(x)>0 else None)
ques_df.dropna(subset=['Tags'], inplace= True)
ques_df.shape

(58872, 5)

## Preprocessing of body and title
1.removing html tags
2.removing punctuation
3.lemmatizing
4.removing stop words

In [34]:
ques_df['Body']=ques_df['Body'].apply( lambda x: BeautifulSoup(x).get_text())

In [35]:
def rem_punctuation(text):
    for punctuation in string.punctuation:
        text=text.replace(punctuation,'')
    return text

In [36]:
ques_df['Title_new']=ques_df['Title'].apply(rem_punctuation)
ques_df['Title_new']=ques_df['Title_new'].str.lower()
ques_df['Title_new']=ques_df['Title_new'].str.split()
ques_df['Title_new'].head()

2                                 [aspnet, site, maps]
3             [function, for, creating, color, wheels]
4    [adding, scripting, functionality, to, net, ap...
5    [should, i, use, nested, classes, in, this, case]
6          [homegrown, consumption, of, web, services]
Name: Title_new, dtype: object

In [37]:
ques_df['Body_new']=ques_df['Body'].apply(rem_punctuation)
ques_df['Body_new']=ques_df['Body_new'].str.lower()
ques_df['Body_new']=ques_df['Body_new'].str.split()
ques_df['Body_new'].head()

2    [has, anyone, got, experience, creating, sqlba...
3    [this, is, something, ive, pseudosolved, many,...
4    [i, have, a, little, game, written, in, c, it,...
5    [i, am, working, on, a, collection, of, classe...
6    [ive, been, writing, a, few, web, services, fo...
Name: Body_new, dtype: object

In [38]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
# from nltk.tokenize.toktok import ToktokTokenizer
lemmatizer=WordNetLemmatizer()
# tokenizer=ToktokTokenizer()

def word_lemmatizer(text):
    lem_text=[lemmatizer.lemmatize(i) for i in text if i not in set(stopwords.words('english'))]
    return lem_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [39]:
ques_df['Title_new']=ques_df['Title_new'].apply(lambda x: word_lemmatizer(x))

In [40]:
a=ques_df['Title_new']

In [41]:
ques_df['Title_new']=a

In [42]:
ques_df['Title_new']

2                                        [aspnet, site, map]
3                         [function, creating, color, wheel]
4          [adding, scripting, functionality, net, applic...
5                                 [use, nested, class, case]
6                     [homegrown, consumption, web, service]
                                 ...                        
1262834                        [stl, list, bad, performance]
1262915                       [use, dict, subset, dataframe]
1263065    [way, use, itertools, python, clean, nested, i...
1263454          [result, data, returned, void, get, broken]
1264205            [exactly, make, type, system, consistent]
Name: Title_new, Length: 58872, dtype: object

In [43]:
ques_df['Body_new']=ques_df['Body_new'].apply(lambda x: word_lemmatizer(x))

In [44]:
ques_df['combined_text'] = ques_df['Title_new'].astype(str) + ' ' + ques_df['Body_new'].astype(str)

Data Preparation

In [47]:
ques_df.drop(columns=['Score','Title', 'Body'])

,Id,Tags,Title_new,Body_new,combined_text
2,120,"[sql, asp.net]","[aspnet, site, map]","[anyone, got, experience, creating, sqlbased, ...","['aspnet', 'site', 'map'] ['anyone', 'got', 'e..."
3,180,[algorithm],"[function, creating, color, wheel]","[something, ive, pseudosolved, many, time, nev...","['function', 'creating', 'color', 'wheel'] ['s..."
4,260,"[c#, .net]","[adding, scripting, functionality, net, applic...","[little, game, written, c, us, database, backe...","['adding', 'scripting', 'functionality', 'net'..."
5,330,[c++],"[use, nested, class, case]","[working, collection, class, used, video, play...","['use', 'nested', 'class', 'case'] ['working',..."
6,470,[.net],"[homegrown, consumption, web, service]","[ive, writing, web, service, net, app, im, rea...","['homegrown', 'consumption', 'web', 'service']..."
...,...,...,...,...,...
1262834,40110020,[c++],"[stl, list, bad, performance]","[supposed, pushback, popfront, method, stl, li...","['stl', 'list', 'bad', 'performance'] ['suppos..."
1262915,40111730,[python],"[use, dict, subset, dataframe]","[say, given, dataframe, column, categorical, d...","['use', 'dict', 'subset', 'dataframe'] ['say',..."
1263065,40115300,[python],"[way, use, itertools, python, clean, nested, i...","[let, say, following, code, 123, b, 246, c, 35...","['way', 'use', 'itertools', 'python', 'clean',..."
1263454,40125310,[c++],"[result, data, returned, void, get, broken]","[working, project, huge, legacy, code, base, t...","['result', 'data', 'returned', 'void', 'get', ..."


In [48]:
# X1=ques_df['Title_new']
# X2= ques_df['Body_new']
X= ques_df['combined_text']
y=ques_df['Tags']

In [49]:
len(X)
# print(X1)

58872

In [50]:
len(y)

58872

In [51]:
from sklearn.preprocessing import MultiLabelBinarizer
multilabel_binarizer= MultiLabelBinarizer()
y_bin= multilabel_binarizer.fit_transform(y)

In [52]:
y_bin

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [53]:
multilabel_binarizer.classes_

array(['.net', 'algorithm', 'android', 'angularjs', 'arrays', 'asp.net',
       'asp.net-mvc', 'c', 'c#', 'c++', 'c++11', 'css', 'database',
       'django', 'eclipse', 'git', 'haskell', 'html', 'html5', 'ios',
       'iphone', 'java', 'javascript', 'jquery', 'json', 'linux',
       'multithreading', 'mysql', 'node.js', 'objective-c', 'osx',
       'performance', 'php', 'python', 'r', 'regex', 'ruby',
       'ruby-on-rails', 'scala', 'spring', 'sql', 'sql-server', 'string',
       'swift', 'unit-testing', 'visual-studio', 'windows', 'wpf',
       'xcode', 'xml'], dtype=object)

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
ques_df['Title_new']=ques_df['Title_new'].astype(str)
ques_df['Body_new']=ques_df['Body_new'].astype(str)
ques_df['combined_text']=ques_df['combined_text'].astype(str)
tfidf= TfidfVectorizer(analyzer= 'word', max_features=1000, ngram_range=(1,2) , encoding='utf-8')
# X1_tfidf= tfidf.fit_transform(ques_df['Title_new'])
# X2_tfidf= tfidf.fit_transform(ques_df['Body_new'])
X_tfidf= tfidf.fit_transform(ques_df['combined_text'])

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X_tfidf, y_bin, test_size=0.2, random_state=0)

In [56]:
X_train.shape

(47097, 1000)

In [57]:
y_train.shape

(47097, 50)

## Classification and Prediction

In [58]:
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, make_scorer, multilabel_confusion_matrix ,confusion_matrix, hamming_loss, accuracy_score, precision_score, recall_score, f1_score

In [59]:
sgd= SGDClassifier()
lr= LogisticRegression(C=10)
svc= LinearSVC()
prec_dict={}
hamloss_dict={}


In [60]:
for classes in [lr, sgd, svc]:
    clf=OneVsRestClassifier(classes)
    clf.fit(X_train, y_train)
    y_pred= clf.predict(X_test)
    ham_loss= hamming_loss(y_test, y_pred)
    prec= precision_score(y_test, y_pred, average= 'weighted')
    recall= recall_score(y_test, y_pred, average='weighted')
    f1= f1_score(y_test, y_pred, average='weighted')
    clsnm= classes.__class__.__name__
    prec_dict[clsnm]=prec
    hamloss_dict[clsnm]=ham_loss
    print('Classifier: ', clsnm)
    print("Hamming Loss: ", ham_loss)
    print('Precision: ', prec)
    print('Recall: ', recall)
    print('F1 score: ', f1)
    print('\n\n')

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Classifier:  LogisticRegression
Hamming Loss:  0.017778343949044585
Precision:  0.7631698312233793
Recall:  0.5116111611161116
F1 score:  0.6070757999684333





/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classifier:  SGDClassifier
Hamming Loss:  0.018771974522292994
Precision:  0.8033896328208527
Recall:  0.39633963396339633
F1 score:  0.5042002012917451



Classifier:  LinearSVC
Hamming Loss:  0.017528662420382167
Precision:  0.7920039500369316
Recall:  0.4902490249024902
F1 score:  0.5940148449590976



